In [51]:
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Flatten
from functions import VM
import os
import numpy as np

# Take the sign names in our collections data

In [52]:
signs = sorted(os.listdir("./Data/")) # take all the signs used in data collection
sign_id = {label: num for num, label in enumerate(signs)} # assign a value for each sign (each sign is assign to its index)

In [12]:
signs[2]

'hi'

In [13]:
sign_id['hello']

1

# Create the our x and y data


In [53]:
sequences, labels = [], [] # sequences represents our x data while labels represents our y data
# create a loop that creates our x and y data
for sign in signs:
    for sequence in range(VM.seq):
        window = []
        for frames in range(VM.seq_lenght):
            res = np.load(os.path.join(VM.DATA_PATH, sign, str(sequence), f"{frames}.npy"))
            window.append(res)
        sequences.append(window)
        labels.append(sign_id[sign])
        
# store our x and y data to a numpy array
x_data = np.array(sequences) 
y_data = to_categorical(labels).astype(int) 

# setup our training and testing partition
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.05)

# Build and train the model


In [55]:
# build the model
model = Sequential()
model.add(LSTM(64, return_sequences="True", activation="relu", input_shape=(30,1662)))
model.add(LSTM(128, return_sequences="True", activation="relu"))
model.add(LSTM(64, return_sequences="False", activation="relu"))
model.add(Flatten(input_shape=(x_train.shape[1:])))
model.add(Dense(32, activation="relu"))
model.add(Dense(64, activation="relu"))
model.add(Dense(np.array(signs).shape[0], activation="softmax"))

In [56]:
model.compile(optimizer="Adam", loss="categorical_crossentropy", metrics=["categorical_accuracy"]) # compiler 

In [57]:
model.fit(x_train, y_train, epochs=80) # run this line when data collection is done (this will train the model)
# we can start the number of epochs to be equal to 175 to avoid over training and add afterwards if needed

Epoch 1/80
9/9 [==============================] - 3s 72ms/step - loss: 2.1957 - categorical_accuracy: 0.1509
Epoch 2/80
9/9 [==============================] - 1s 80ms/step - loss: 1.8867 - categorical_accuracy: 0.1965
Epoch 3/80
9/9 [==============================] - 1s 77ms/step - loss: 1.8966 - categorical_accuracy: 0.1965
Epoch 4/80
9/9 [==============================] - 1s 72ms/step - loss: 1.7201 - categorical_accuracy: 0.2632
Epoch 5/80
9/9 [==============================] - 1s 69ms/step - loss: 1.7976 - categorical_accuracy: 0.1754
Epoch 6/80
9/9 [==============================] - 1s 71ms/step - loss: 1.8681 - categorical_accuracy: 0.2105
Epoch 7/80
9/9 [==============================] - 1s 70ms/step - loss: 1.8152 - categorical_accuracy: 0.1895
Epoch 8/80
9/9 [==============================] - 1s 68ms/step - loss: 1.8739 - categorical_accuracy: 0.2246
Epoch 9/80
9/9 [==============================] - 1s 69ms/step - loss: 1.7899 - categorical_accuracy: 0.1930
Epoch 10/80
9/9 [==

9/9 [==============================] - 1s 69ms/step - loss: 0.3000 - categorical_accuracy: 0.8561
Epoch 76/80
9/9 [==============================] - 1s 68ms/step - loss: 0.2729 - categorical_accuracy: 0.8772
Epoch 77/80
9/9 [==============================] - 1s 66ms/step - loss: 0.2521 - categorical_accuracy: 0.8667
Epoch 78/80
9/9 [==============================] - 1s 67ms/step - loss: 0.2660 - categorical_accuracy: 0.8491
Epoch 79/80
9/9 [==============================] - 1s 68ms/step - loss: 0.2480 - categorical_accuracy: 0.8877
Epoch 80/80
9/9 [==============================] - 1s 69ms/step - loss: 0.2618 - categorical_accuracy: 0.8807


In [54]:
del model

In [30]:
model.summary() # summary of our model

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_9 (LSTM)               (None, 30, 64)            442112    
                                                                 
 lstm_10 (LSTM)              (None, 30, 128)           98816     
                                                                 
 lstm_11 (LSTM)              (None, 30, 64)            49408     
                                                                 
 flatten_3 (Flatten)         (None, 1920)              0         
                                                                 
 dense_9 (Dense)             (None, 32)                61472     
                                                                 
 dense_10 (Dense)            (None, 64)                2112      
                                                                 
 dense_11 (Dense)            (None, 13)               

In [58]:
res = model.predict(x_test)

1/1 [==============================] - 1s 593ms/step


In [59]:
signs[np.argmax(res[5])]

'thank you'

In [60]:
signs[np.argmax(y_test[5])]

'thank you'

# save weights


In [61]:
model.save("VertoMotus_MLmodel4.h5") # save model

In [89]:
model.save('VertoMotus_MLmodel(2.0- 97).model')

INFO:tensorflow:Assets written to: VertoMotus_MLmodel(2.0- 97).model\assets


In [ ]:
model = tf.keras.models.load_model("./VertoMotus_MLmodel4.h5")
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_ops = [
  tf.lite.OpsSet.TFLITE_BUILTINS, # enable TensorFlow Lite ops.
  tf.lite.OpsSet.SELECT_TF_OPS # enable TensorFlow ops.
]
tflite = converter.convert()

open("./LiteML3.tflite", "wb").write(tflite)

INFO:tensorflow:Assets written to: C:\Users\Baron\AppData\Local\Temp\tmp0c6_gw35\assets


INFO:tensorflow:Assets written to: C:\Users\Baron\AppData\Local\Temp\tmp0c6_gw35\assets


In [38]:
interpreter = tf.lite.Interpreter("LiteML.tflite")

inp = interpreter.get_input_details()
out = interpreter.get_output_details()

interpreter.set_tensor(inp[0]["index"], np.expand_dims(sequences,axis=0))
interpreter.invoke()
                
                #for i in sequence[-30:]:
output_data = interpreter.get_tensor(out[0]["index"])
                
highest = np.argmax(output_data)
word = VM.get_key(signs, highest)
print(output_data.shape)
print(word)


ValueError: Cannot set tensor: Got value of type FLOAT64 but expected type FLOAT32 for input 0, name: serving_default_lstm_9_input:0 